In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.layers import Bidirectional, LSTM, Dropout, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import plot_model

from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor

In [11]:
data = pd.read_csv('/Users/yashwanthkaruparthi/Documents/Documents1/acad/sem7/design project/implementation/data/solar_weather.csv', header=0, infer_datetime_format=True, parse_dates=['Time'], index_col=['Time'])

data = data[(data.index.month.isin([5,6,7])) & (data.index.year == 2021)]
# dataset = data[['Energy delta[Wh]', 'GHI', 'temp', 'pressure', 'humidity']]
dataset = data[['Energy delta[Wh]', 'GHI']]
X = dataset.iloc[:, 1:].values
y = dataset.iloc[:, 0].values

train_size = int(len(X) * 0.8)
test_size = len(X) - train_size
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[test_size:]

/var/folders/n5/mlsmdgns0kqfybw1r9qm4n5w0000gp/T/ipykernel_37268/2821328646.py:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  data = pd.read_csv('/Users/yashwanthkaruparthi/Documents/Documents1/acad/sem7/design project/implementation/data/solar_weather.csv', header=0, infer_datetime_format=True, parse_dates=['Time'], index_col=['Time'])


In [12]:
evaluation_results = []

# Modified evaluate_model function to store results in evaluation_results
def evaluate_model(y_true, y_pred, model_name, dataset_type):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

    # Append metrics to evaluation_results
    evaluation_results.append({
        'Model': model_name,
        'Dataset': dataset_type,
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'R²': r2
    })

In [27]:
train_size = int(len(X) * 0.8)
test_size = len(X) - train_size
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[test_size:]

catboost = CatBoostRegressor(n_estimators=100, learning_rate=0.001, depth=5, verbose=0)
catboost.fit(X_train, y_train.ravel())

catboost_output_train = catboost.predict(X_train)
catboost_output_test = catboost.predict(X_test)

scaler = MinMaxScaler()
catboost_output_train_scaled = scaler.fit_transform(catboost_output_train.reshape(-1, 1))
catboost_output_test_scaled = scaler.transform(catboost_output_test.reshape(-1, 1))

def create_dataset(dataset, time_step):
    dataX, dataY = [], []
    for i in range(len(dataset) - time_step):
        a = dataset[i:(i + time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

time_step = 24

X_train_bilstm_catboost, y_train_bilstm_catboost = create_dataset(catboost_output_train_scaled, time_step)
X_test_bilstm_catboost, y_test_bilstm_catboost = create_dataset(catboost_output_test_scaled, time_step)

X_train_bilstm_catboost = X_train_bilstm_catboost.reshape(X_train_bilstm_catboost.shape[0], time_step, 1)
X_test_bilstm_catboost = X_test_bilstm_catboost.reshape(X_test_bilstm_catboost.shape[0], time_step, 1)

model = Sequential()
model.add(Bidirectional(LSTM(250, activation='relu', return_sequences=True, input_shape=(time_step, 1))))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(250, activation='relu', return_sequences=False)))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

mse_loss = tf.keras.losses.MeanSquaredError()


def train_with_teacher_forcing(model, X_train, y_train, batch_size=64, epochs=10, time_step=24):
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        for i in range(0, len(X_train) - batch_size, batch_size):
            # Select batch
            X_batch = X_train[i:i + batch_size]
            y_batch = y_train[i:i + batch_size]
            
            # Initialize predictions array to match y_batch dimensions
            predictions = np.zeros_like(y_batch)

            # Perform teacher forcing for each time step
            for t in range(time_step):
                # Use actual values for teacher forcing up to the current time step
                if t > 0:
                    # Update each time step in X_batch using teacher-forced value
                    X_batch[:, t, 0] = y_batch  # Corrected indexing

                # Train the model on the modified X_batch
                with tf.GradientTape() as tape:
                    pred = model(X_batch, training=True)
                    predictions = pred[:, 0]  # Update predictions array
                    loss = mse_loss(y_batch, predictions)  # Adjusted indexing for compatibility

                # Update model weights
                gradients = tape.gradient(loss, model.trainable_variables)
                model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        print(f"Loss at epoch {epoch+1}: {loss.numpy()}")

print("Training BiLSTM with Teacher Forcing on CatBoost outputs...")
train_with_teacher_forcing(model, X_train_bilstm_catboost, y_train_bilstm_catboost, batch_size=64, epochs=10)

# Step 7: Predict with BiLSTM (without teacher forcing in prediction phase)
y_pred_bilstm_catboost_scaled = model.predict(X_test_bilstm_catboost)
y_pred_bilstm_catboost = scaler.inverse_transform(y_pred_bilstm_catboost_scaled)

# Step 8: Rescale y_test and Evaluate
y_test_bilstm_catboost_actual_rescaled = scaler.inverse_transform(y_test_bilstm_catboost.reshape(-1, 1))
evaluate_model(y_test_bilstm_catboost_actual_rescaled, y_pred_bilstm_catboost, 'BiLSTM (CatBoost)', 'Testing')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Training BiLSTM with Teacher Forcing on CatBoost outputs...
Epoch 1/10


KeyboardInterrupt: 

In [13]:
def create_teacher_forcing_dataset(X, y, time_step=24):
    """
    Efficiently create teacher forcing dataset
    
    Args:
    X (np.array): Input features
    y (np.array): Target values
    time_step (int): Number of time steps to look back
    
    Returns:
    Tuple of (X_teacher, y_teacher)
    """
    X_teacher, y_teacher = [], []
    for i in range(len(X) - time_step):
        X_seq = X[i:i+time_step]
        y_seq = y[i:i+time_step]
        X_teacher.append(X_seq)
        y_teacher.append(y_seq)

    X_teacher = np.array(X_teacher).reshape(-1, time_step, 1)
    y_teacher = np.array(y_teacher).reshape(-1, 1)
    
    return np.array(X_teacher), np.array(y_teacher)

def create_optimized_model(time_step, feature_dim=1):
    
    model = Sequential([
        Bidirectional(LSTM(250, 
                           return_sequences=True, 
                           activation='relu', 
                           input_shape=(time_step, feature_dim))),
        Dropout(0.2),
        Bidirectional(LSTM(250, 
                           return_sequences=True, 
                           activation='relu')),
        Dropout(0.2),
        Dense(1, activation='sigmoid')  # Output layer
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mean_squared_error')
    
    return model

# Optimized Training Function
def train_optimized_teacher_forcing(X, y, time_step=24, epochs=10, batch_size=64):
    # Create teacher forcing dataset
    X_teacher, y_teacher = create_teacher_forcing_dataset(X, y, time_step)
    
    # Create model
    model = create_optimized_model(time_step)
    
    # Early stopping and model checkpoint
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='loss', 
        patience=3, 
        restore_best_weights=True
    )
    
    # Tensorboard for monitoring
    log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir, histogram_freq=1
    )
    
    # Train with callbacks
    history = model.fit(
        X_teacher, y_teacher,
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[early_stopping, tensorboard_callback],
        verbose=1
    )
    
    return model, history

# Usage

catboost = CatBoostRegressor(n_estimators=100, learning_rate=0.001, depth=5, verbose=0)
catboost.fit(X_train, y_train.ravel())

catboost_output_train = catboost.predict(X_train)
catboost_output_test = catboost.predict(X_test)

scaler = MinMaxScaler()
catboost_output_train_scaled = scaler.fit_transform(catboost_output_train.reshape(-1, 1))
catboost_output_test_scaled = scaler.transform(catboost_output_test.reshape(-1, 1))

optimized_model, training_history = train_optimized_teacher_forcing(
    catboost_output_train_scaled, 
    catboost_output_test_scaled
)

y_pred_scaled = optimized_model.predict(X_test_bilstm_catboost)
y_pred_bilstm_catboost = scaler.inverse_transform(y_pred_scaled)

y_test_bilstm_catboost_actual_rescaled = scaler.inverse_transform(y_test_bilstm_catboost.reshape(-1, 1))
evaluate_model(y_test_bilstm_catboost_actual_rescaled, y_pred_bilstm_catboost, 'BiLSTM (CatBoost) Optimized', 'Testing')

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (7041,) + inhomogeneous part.

In [ ]:
import numpy as np
import tensorflow as tf
from catboost import CatBoostRegressor
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dropout, Dense

# Function to create datasets for teacher forcing
def create_teacher_forcing_dataset(X, y, time_step=24):
    X_teacher, y_teacher = [], []
    for i in range(len(X) - time_step):
        X_seq = X[i:i + time_step]
        y_seq = y[i + time_step]  # Only the next value is the target
        X_teacher.append(X_seq)
        y_teacher.append(y_seq)

    X_teacher = np.array(X_teacher).reshape(-1, time_step, 1)
    y_teacher = np.array(y_teacher).reshape(-1, 1)

    return np.array(X_teacher), np.array(y_teacher)

# Function to create the BiLSTM model
def create_bilstm_model(time_step, feature_dim=1):
    model = Sequential([
        Bidirectional(LSTM(250, return_sequences=True, activation='relu', input_shape=(time_step, feature_dim))),
        Dropout(0.2),
        Bidirectional(LSTM(250, return_sequences=False, activation='relu')),
        Dropout(0.2),
        Dense(1)  # Output layer should output shape (batch_size, 1)
    ])
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    return model

# Optimized Training Function with Teacher Forcing
def train_with_teacher_forcing(X, y, time_step=24, epochs=10, batch_size=64):
    # Create teacher forcing dataset
    X_teacher, y_teacher = create_teacher_forcing_dataset(X, y, time_step)

    # Create model
    model = create_bilstm_model(time_step)

    # Store predictions for the entire sequence
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        # Iterate over batches
        for i in range(0, len(X_teacher), batch_size):
            X_batch = X_teacher[i:i + batch_size]
            y_batch = y_teacher[i:i + batch_size]

            # Initialize an empty array for predictions
            predictions = np.zeros((X_batch.shape[0], time_step, 1))

            # Perform teacher forcing
            for t in range(time_step):
                # Use actual values for teacher forcing up to the current time step
                if t == 0:
                    current_input = X_batch
                else:
                    # Concatenate previous predictions along the last dimension
                    current_input = np.concatenate((X_batch[:, :t, :], predictions[:, :t, :]), axis=1)

                # Predict for the current input
                pred = model(current_input)
                predictions[:, t, 0] = pred.numpy().flatten()  # Store predictions for teacher forcing

            # Train the model on the entire sequence of predictions
            model.fit(X_batch, predictions[:, -1, :], epochs=1, batch_size=batch_size, verbose=0)

    return model

# Example usage
# Prepare your data (X, y) here

# Step 2: Train a GBDT model (e.g., CatBoost)
catboost = CatBoostRegressor(n_estimators=100, learning_rate=0.01, depth=5)
catboost.fit(X_train, y_train.ravel())

# Step 3: Get predictions from the GBDT model
catboost_output_train = catboost.predict(X_train)
catboost_output_test = catboost.predict(X_test)

# Step 4: Scale the predictions
scaler = MinMaxScaler()
catboost_output_train_scaled = scaler.fit_transform(catboost_output_train.reshape(-1, 1))
catboost_output_test_scaled = scaler.transform(catboost_output_test.reshape(-1, 1))

# Step 5: Train the BiLSTM model with teacher forcing
bilstm_model = train_with_teacher_forcing(catboost_output_train_scaled, y_train, time_step=24, epochs=10, batch_size=64)

# Step 6: Make predictions with the trained BiLSTM model
y_pred_scaled = bilstm_model.predict(catboost_output_test_scaled.reshape(-1, 24, 1))
y_pred = scaler.inverse_transform(y_pred_scaled)

# Evaluate model (implement your own evaluation function)
# evaluate_model(y_test, y_pred)


0:	learn: 1160.0621403	total: 3.37ms	remaining: 334ms
1:	learn: 1150.0888094	total: 4ms	remaining: 196ms
2:	learn: 1140.3052419	total: 4.41ms	remaining: 143ms
3:	learn: 1130.6651913	total: 4.85ms	remaining: 116ms
4:	learn: 1121.0253091	total: 5.31ms	remaining: 101ms
5:	learn: 1111.4080169	total: 5.79ms	remaining: 90.8ms
6:	learn: 1101.9343300	total: 6.26ms	remaining: 83.2ms
7:	learn: 1092.6330847	total: 6.75ms	remaining: 77.6ms
8:	learn: 1083.3797465	total: 7.22ms	remaining: 73ms
9:	learn: 1074.2398394	total: 7.86ms	remaining: 70.7ms
10:	learn: 1065.1136438	total: 8.36ms	remaining: 67.7ms
11:	learn: 1056.1503391	total: 8.86ms	remaining: 65ms
12:	learn: 1047.3234800	total: 9.37ms	remaining: 62.7ms
13:	learn: 1038.5049395	total: 9.91ms	remaining: 60.9ms
14:	learn: 1029.7879522	total: 10.5ms	remaining: 59.3ms
15:	learn: 1021.1811491	total: 11ms	remaining: 57.6ms
16:	learn: 1012.7667430	total: 11.5ms	remaining: 56ms
17:	learn: 1004.3842815	total: 12ms	remaining: 54.7ms
18:	learn: 996.15514

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [ ]:
results_df = pd.DataFrame(evaluation_results)


In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(y_test_bilstm_catboost_actual_rescaled, y_pred_bilstm_catboost)
plt.plot([y_test_bilstm_catboost_actual_rescaled.min(), y_test_bilstm_catboost_actual_rescaled.max()], 
         [y_test_bilstm_catboost_actual_rescaled.min(), y_test_bilstm_catboost_actual_rescaled.max()], 
         'r--', lw=2)
plt.title('Actual vs Predicted Values')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.show()